In [ ]:
!pip install crewai crewai-tools langchain-groq python-dotenv gradio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.4/80.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 4.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of instructor to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 6.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opentelemetry-exporter-otlp-proto-grpc to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 710.0/710.0 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.8/766.8 kB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# https://pypi.python.org/pypi/libarchive
!apt-get -qq install -y libarchive-dev && pip install -U libarchive
import libarchive

Selecting previously unselected package libarchive-dev:amd64.
(Reading database ... 117528 files and directories currently installed.)
Preparing to unpack .../libarchive-dev_3.6.0-1ubuntu1.5_amd64.deb ...
Unpacking libarchive-dev:amd64 (3.6.0-1ubuntu1.5) ...
Setting up libarchive-dev:amd64 (3.6.0-1ubuntu1.5) ...
Processing triggers for man-db (2.10.2-1) ...
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 3.0 MB/s eta 0:00:00
  Created wheel for libarchive: filename=libarchive-0.4.7-py3-none-any.whl size=31629 sha256=cfe6b0eaca9e765ed0d55cb96e23c66fc4a6c36c5573ea9244745e4d03216db7
  Stored in directory: /root/.cache/pip/wheels/29/20/ab/f101da7b245b996aa097685ef742243725ea6150f5b3b6d9ed
Successfully built libarchive


In [ ]:
# https://pypi.python.org/pypi/pydot
!apt-get -qq install -y graphviz && pip install pydot
import pydot

In [ ]:
!pip install cartopy
import cartopy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 83.6 MB/s eta 0:00:00


In [ ]:
import os
os.environ["GROQ_API_KEY"] = "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"  # Yahan apna Groq key dalo


In [ ]:
import requests

def get_aqi(city="Delhi"):
    # FREE Open-Meteo API (no key needed)
    if city == "Delhi":
        lat, lon = 28.6139, 77.2090
    elif city == "Lucknow":
        lat, lon = 26.8467, 80.9462
    else:
        lat, lon = 28.6139, 77.2090

    url = f"https://air-quality-api.open-meteo.com/v1/air-quality?latitude={lat}&longitude={lon}&hourly=pm10,pm2_5"
    data = requests.get(url).json()
    pm25 = data['hourly']['pm2_5'][0]

    if pm25 > 100:
        level = "DANGEROUS 😷"
    elif pm25 > 50:
        level = "POOR ☠️"
    else:
        level = "GOOD ✅"

    return f"{city} PM2.5: {pm25} µg/m³ - {level}"


In [ ]:
from crewai import Agent, Task, Crew, Process
from langchain_groq import ChatGroq
import os

# Set a dummy OPENAI_API_KEY to bypass CrewAI's internal check
os.environ["OPENAI_API_KEY"] = "dummy"

# ✅ Groq model (FAST + FREE)
llm = ChatGroq(model="llama3.1-8b-instant")

# 🧑‍💼 AGENT 1: Pollution Expert
analyst = Agent(
    role="Pollution Analyst",
    goal="Check air quality and health risks",
    backstory="IIT environmental expert",
    llm=llm,
    verbose=True
)

# 👨‍⚕️ AGENT 2: Doctor
doctor = Agent(
    role="Health Advisor",
    goal="Give safety tips for pollution",
    backstory="Respiratory disease specialist",
    llm=llm,
    verbose=True
)

print("✅ Agents ready!")

✅ Agents ready!


In [ ]:
import gradio as gr
import requests
from langchain_groq import ChatGroq
import os

def air_safety_agent(city):
    try:
        # Safe city coordinates
        cities = {
            "Delhi": (28.61, 77.20),
            "Lucknow": (26.84, 80.94),
            "Mathura": (27.49, 77.68),
            "Agra": (27.18, 78.01),
            "Noida": (28.53, 77.01),
        }

        if city not in cities:
            return "❌ City not supported! Try: Delhi, Lucknow, Mathura, Agra,Noida"

        lat, lon = cities[city]

        # API call with error handling
        url = f"https://air-quality-api.open-meteo.com/v1/air-quality"
        params = {"latitude": lat, "longitude": lon, "hourly": "pm2_5"}
        response = requests.get(url, params=params, timeout=10)

        if response.status_code != 200:
            pm25 = 75.0  # Default value
        else:
            data = response.json()
            pm25 = data['hourly']['pm2_5'][0] if data['hourly']['pm2_5'] else 75.0

        # Groq AI with error handling
        try:
            llm = ChatGroq(model="llama3.1-8b-instant", temperature=0)
            prompt = f"""
            City: {city}, PM2.5: {pm25:.1f}
            College students ke liye Hindi mein safety advice do:
            1. Risk level (Low/Medium/High)
            2. Mask type
            3. Avoid karne wali activities
            4. Emergency symptoms
            Short rakhna (100 words max)
            """
            result = llm.invoke(prompt)
            ai_advice = result.content
        except:
            ai_advice = "N95 Wear mask.\nOutdoor avoid activities. \nIf you have a breathing problem, consult a doctor."


        return f"""
🚨 {city.upper()} AIR QUALITY REPORT 🚨
PM2.5: {pm25:.1f} µg/m³

🎯 AI SAFETY ADVICE:
{ai_advice}

🏆 GLA Techनव्या'26 - Agent-A-Thon Demo
#CampusSafety #AirQualityAgent
        """

    except Exception as e:
        return f"❌ Error: {str(e)}"

# Perfect UI
demo = gr.Interface(
    fn=air_safety_agent,
    inputs=gr.Textbox(
        label="🌆Enter City ",
        value="",
        placeholder="Delhi, Lucknow, Mathura, Agra,Noida"
    ),
    outputs=gr.Textbox(label="🤖 AI Safety Report", lines=20),
    title="🏆 GLA AIR SAFETY AGENT v2.0 🏆",
    description="Real-time pollution monitoring + Multi-Agent Safety System"
)

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://30771da2cfba4833d2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
